# Church SaaS Super Chatbot

**profe-ssor** · Week 2 community contribution

A chatbot that reads your **Church SaaS project URL** and answers any question about it using the page content as context.

1. Paste your project URL and click **Load project**.
2. Ask anything about the project in the chat.

In [12]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import requests
from bs4 import BeautifulSoup

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
openrouter_api_key = os.getenv('OPENROUTER_API_KEY')

openai = OpenAI()
openrouter_url = "https://openrouter.ai/api/v1"
openrouter = OpenAI(api_key=openrouter_api_key, base_url=openrouter_url)
MODEL = 'gpt-4.1-mini'
OPENROUTER_MODEL = 'openai/gpt-4o-mini'

print("OpenAI:", "ready" if openai_api_key else "not set")
print("OpenRouter:", "ready" if openrouter_api_key else "not set (optional)")

OpenAI: ready
OpenRouter: ready


## Fetch website content

We scrape the URL (title + body text) and keep a sensible character limit so the LLM has enough context without overflowing.

In [13]:
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}
MAX_CHARS = 12_000  # enough context for a "super" chatbot

def fetch_site_content(url):
    """Fetch title and main text from URL; return (content, error_message)."""
    if not url or not url.strip().startswith("http"):
        return None, "Please enter a valid URL starting with http:// or https://"
    try:
        r = requests.get(url.strip(), headers=HEADERS, timeout=15)
        r.raise_for_status()
        soup = BeautifulSoup(r.content, "html.parser")
        title = soup.title.string if soup.title else "No title"
        if soup.body:
            for tag in soup.body(["script", "style", "img", "input"]):
                tag.decompose()
            text = soup.body.get_text(separator="\n", strip=True)
        else:
            text = ""
        content = (title + "\n\n" + text)[:MAX_CHARS]
        return content, None
    except requests.RequestException as e:
        return None, f"Could not fetch URL: {e}"
    except Exception as e:
        return None, f"Error: {e}"

In [14]:
# Global cache: content loaded from the URL (used by the chat)
site_content = ""
site_url = ""

## Chat logic

The assistant answers from the loaded site content. If nothing is loaded yet, it asks the user to load a URL.

In [15]:
def get_system_prompt():
    if not site_content:
        return (
            "You are a helpful assistant for a Church SaaS project. "
            "No project content has been loaded yet. "
            "Politely ask the user to paste their project URL and click 'Load project' first."
        )
    return (
        "You are a helpful assistant for a Church SaaS project. "
        "Answer questions **only** using the following content from the project URL. "
        "If the answer is not in the content, say so and keep the reply brief.\n\n"
        "--- Project content ---\n" + site_content + "\n--- End ---"
    )

def chat(message, history):
    history_formatted = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = (
        [{"role": "system", "content": get_system_prompt()}]
        + history_formatted
        + [{"role": "user", "content": message}]
    )
    if openrouter_api_key:
        r = openrouter.chat.completions.create(model=OPENROUTER_MODEL, messages=messages)
    else:
        r = openai.chat.completions.create(model=MODEL, messages=messages)
    return r.choices[0].message.content

## Load-URL callback and Gradio UI

When the user clicks **Load project**, we fetch the URL and store it in `site_content`. The chat then uses that content to answer.

In [16]:
def load_project(url):
    global site_content, site_url
    content, err = fetch_site_content(url)
    if err:
        return err
    site_content = content
    site_url = url.strip()
    return f"Loaded {len(site_content)} characters from {site_url}. You can ask questions below."

with gr.Blocks(title="Church SaaS Super Chatbot", theme=gr.themes.Soft()) as demo:
    gr.Markdown("## Church SaaS Super Chatbot")
    gr.Markdown("Paste your Church SaaS project URL, click **Load project**, then ask anything about it.")
    with gr.Row():
        url_input = gr.Textbox(
            label="Project URL",
            placeholder="https://your-church-saas-project.com",
            scale=4,
        )
        load_btn = gr.Button("Load project", variant="primary", scale=1)
    status = gr.Textbox(label="Status", interactive=False)
    load_btn.click(fn=load_project, inputs=url_input, outputs=status)
    gr.Markdown("### Ask about your project")
    chatbot = gr.Chatbot(type="messages", label="Chat")
    with gr.Row():
        textbox = gr.Textbox(placeholder="Type your question here...", show_label=False, scale=7, container=False)
        submit_btn = gr.Button("Send", variant="primary", scale=1)
    def chat_ui(message, messages):
        if not (message or str(message).strip()):
            return messages if messages else [], ""
        messages = list(messages) if messages else []
        messages.append({"role": "user", "content": message})
        reply = chat(message, messages[:-1])
        messages.append({"role": "assistant", "content": reply})
        return messages, ""
    submit_btn.click(fn=chat_ui, inputs=[textbox, chatbot], outputs=[chatbot, textbox], trigger_mode="multiple")
    textbox.submit(fn=chat_ui, inputs=[textbox, chatbot], outputs=[chatbot, textbox], trigger_mode="multiple")

demo.launch()

* Running on local URL:  http://127.0.0.1:7891
* To create a public link, set `share=True` in `launch()`.
